In [1]:
#Multiple inheritance

In [2]:
#Множественное наследование от обычного отличается тем, что наследуемый класс имеет более 2 родителей, то есть 
# наследует более 1 класса

In [6]:
class Animal():
    def die(self):
        print('bye-bye')
        self.health = 0
        
class Carnivour():
    def hunt(self):
        print('eating')
        self.satiety = 100
        
class Dog(Animal, Carnivour): #непосредственно множестввенное наследование 
    def bark(self): 
        print('woof-woof')
dog = Dog()
dog.bark()
dog.hunt()
dog.die()
print(dog.health, dog.satiety)

woof-woof
eating
bye-bye
0 100


In [7]:
#проблемки

#deadly diamond of death (смертельный ромб)
class Animal():
    def set_health(self, health):
        print('set in animal')
class Carnivour(Animal): #наследник
    def set_health(self, health): #переопределение метода 
        print('set in carnivour')

class Mamal(Animal): #наследник 
    def set_health(self, health):
        print('set in mamal')
        
class Dog(Mamal, Carnivour): #наследник наследников
    pass

#Получился "смертельный ромб" (по диаграмме наследования Dog наследуется от Mamal и Carnivour, которые наследуются 
# от Animal) - получается ромб

In [10]:
dog = Dog()
dog.set_health(90) #out: set in mamal, т к !ВАЖЕН порядок наследования (слева направо) (в аргументах наследника)

set an mamal


In [16]:
#случай, когда dog также определяет set_health(), но хочет вызвать код из базовых классов 

class Animal():
    def set_health(self, health):
        print('set in animal')

class Carnivour(Animal): 
    def set_health(self, health): 
        print('set in carnivour')

class Mamal(Animal): 
    def set_health(self, health):
        print('set in mamal')
        
class Dog(Mamal, Carnivour):
    def set_health(self, health):
        Mamal.set_health(self, health)
        Carnivour.set_health(self, health)
        Animal.set_health(self, health)
        
        print('set in dog')

dog = Dog()
dog.set_health(213123123123123123)

set in mamal
set in carnivour
set in animal
set in dog


In [17]:
#но любой из классов такой иерархии также может нуждаться в вызове реализации базового класса 

#например пусть каждый из классов в иерархии хочет вызвать метод из родительского, конкретно здесь Carnivour и Mamal
# будут инициализировать Animal

class Animal():
    def set_health(self, health):
        print('set in animal')

class Carnivour(Animal): 
    def set_health(self, health): 
        Animal.set_health(self, health)#
        print('set in carnivour')

class Mamal(Animal): 
    def set_health(self, health):
        Animal.set_health(self, health)#
        print('set in mamal')
        
class Dog(Mamal, Carnivour):
    def set_health(self, health):
        Mamal.set_health(self, health)
        Carnivour.set_health(self, health)
        #Animal.set_health(self, health) - эта строчка теперь лишняя
        
        print('set in dog')
        
dog = Dog()
dog.set_health(10213123)

set in animal
set in mamal
set in animal
set in carnivour
set in dog


In [18]:
# тут 'set in animal' логично вызывается два раза, то есть самый глубокий родитель инициализируется дважды 
#!В решении этой проблемы нам поможет функция super()., которая дает гарантии последовательности вызовов 

#super() сама инициализирует базовый класс, синтаксис '#'
class Animal():
    def set_health(self, health):
        print('set in animal')

class Carnivour(Animal): 
    def set_health(self, health): 
        super().set_health(health)#
        print('set in carnivour')

class Mamal(Animal): 
    def set_health(self, health):
        super().set_health(health)#
        print('set in mamal')
        
class Dog(Mamal, Carnivour):
    def set_health(self, health):
    
        super().set_health(health)
        
        #Mamal.set_health(self, health)
        #Carnivour.set_health(self, health) (((super сама обращаетя к обоим родителям)))
        
        print('set in dog')
        
dog = Dog()
dog.set_health(10213123)

set in animal
set in carnivour
set in mamal
set in dog


In [19]:
#Как правило super используется внутри конструктора базового класса (т е когда наследник инициализирует родителя, а 
# именно его инструментал (__init__))

#!то что мы исследовали (super) - MRO (Method Resolution Order) - механизм, определющий каким образом должна 
# разрешаться последовательность вызовов методов, начиная с Py3 мы полагаемся на функцию super(), которая гарантирует 
# последовательность вызовов, а именно !снизу вверх слева направо;
#Слева направо, потому что учитывается порядок наследования, конкретно у нас в примере::

#mro of Dog(слева-направо): Dog -> Mammal -> Carnivour -> Animal , но вывод происходит в обратную сторону (сначала 
# самый глубокий) (((здесь ВАЖЕН порядок вызова, все элементарно)))

In [20]:
#Если есть необходимость инициализировать базовый класс, следует всегда использовать super() (чтобы не сломать код)

In [24]:
class Animal:
    
    def __init__(self):
        self.health = 100
        
    def hit(self, damage):
        self.health -= damage
        
class Carnivour(Animal):
    
    def __init__(self):
        self.legs = 4 #мы не вызываем конструктор базового, а просто добавляем атрибут наследнику 
        
c = Carnivour()

#c.hit(10);
#print(c.health) - ошибка: 'Carnivour' object has no attribute 'health'   , поскольку мы не вызвали __init__ базового

In [33]:
#мы сделаем это, конечно же, с помощью супер

class Animal:
    
    def __init__(self):
        self.health = 100
        
    def hit(self, damage):
        self.health -= damage
        
class Carnivour(Animal):

    def __init__(self):
        super().__init__() #!вызываем инит внутри методов а не вне их (передаем инструментал в инструментал)
        self.legs = 4 
        
c = Carnivour()
c.hit(1)
print(c.health)

99


In [ ]:
#mix-in бла бла бла бла бла бла бла бла